In [1]:
import numpy as np
import math
from hw6_dataload import LFD_Data
from hw6_nlt import LinRegNLT2

# HW 7
## Validation
**Given :** Linear Regression with nonlinear transformation.


&Phi;(x) = (1, x<sub>1</sub>, x<sub>2</sub>, x<sub>1</sub><sup>, 2</sup>, x<sub>2</sub><sup>2</sup>, x<sub>1</sub>x<sub>2</sub>, |x<sub>1</sub> - x<sub>2</sub>|, |x<sub>1</sub> + x<sub>2</sub>|)


As this transformation is the same as the one from HW6 (as well as the datasets), I will reuse the classes from the last homework.

In [4]:
rwd_train = "hw6_train.dta"
rwd_test = "hw6_test.dta"
l_reg = math.pow(10.0, -3) #actually, doesn't use regularization but just copying over from last hw anyways.

# load data from external files and init
rwd_data = LFD_Data(rwd_train, rwd_test)
rwd_algo = LinRegNLT2(rwd_data.dim, 7, l_reg)

#editing from hw6 to make more flexible
def rwd_print_error(algo,valid_X, valid_Y, out_X, out_Y):
    #ein = algo.calc_error(in_X, in_Y)
    e_valid = algo.calc_error(valid_X, valid_Y)
    eout = algo.calc_error(out_X, out_Y)
    print("E_valid: %f, E_out: %f" % (e_valid, eout))
    
print("=== LinReg with NLT using the first 25 examples for training (last 10 for validation) ===")
my_k = np.arange(3,8) #up through which cols (0-idx to use for training)
#train without regularization
for k in my_k:
    print("k = %d :" % k)
    rwd_algo.set_k(k)
    rwd_algo.train(rwd_data.train_X[:25,:], rwd_data.train_Y[:25])
    rwd_print_error(rwd_algo, rwd_data.train_X[25:,:], rwd_data.train_Y[25:], rwd_data.test_X, rwd_data.test_Y)

print("")
print("=== LinReg with NLT using the last 10 examples for training (first 25 for validation) ===")
my_k = np.arange(3,8) #up through which cols (0-idx to use for training)
#train without regularization
for k in my_k:
    print("k = %d :" % k)
    rwd_algo.set_k(k)
    rwd_algo.train(rwd_data.train_X[25:,:], rwd_data.train_Y[25:])
    rwd_print_error(rwd_algo, rwd_data.train_X[:25,:], rwd_data.train_Y[:25], rwd_data.test_X, rwd_data.test_Y)

=== LinReg with NLT using the first 25 examples for training (last 10 for validation) ===
k = 3 :
E_valid: 0.300000, E_out: 0.420000
k = 4 :
E_valid: 0.500000, E_out: 0.416000
k = 5 :
E_valid: 0.200000, E_out: 0.188000
k = 6 :
E_valid: 0.000000, E_out: 0.084000
k = 7 :
E_valid: 0.100000, E_out: 0.072000

=== LinReg with NLT using the last 10 examples for training (first 25 for validation) ===
k = 3 :
E_valid: 0.280000, E_out: 0.396000
k = 4 :
E_valid: 0.360000, E_out: 0.388000
k = 5 :
E_valid: 0.200000, E_out: 0.284000
k = 6 :
E_valid: 0.080000, E_out: 0.192000
k = 7 :
E_valid: 0.120000, E_out: 0.196000


## Validation Bias

**Want :** Expected values of e, e1, and e2 given that e1 and e2 are independent, uniformly-distributed random variables over [0,1] and e = min(e1, e2)

In [7]:
vb_n = 100000 #number of random numbers to generate for each e1 and e2
e1 = np.random.uniform(0,1, vb_n)
e2 = np.random.uniform(0,1, vb_n)
e = np.minimum(e1, e2)

ev_e1 = np.average(e1)
ev_e2 = np.average(e2)
ev_e = np.average(e)

print("Expected values: ")
print("e: %f" % ev_e)
print("e1: %f" % ev_e1)
print("e2: %f" % ev_e2)

Expected values: 
e: 0.333768
e1: 0.501344
e2: 0.499188


## Cross Validation

**Given :**
- data points (x,y): (-1,0), (&rho;, 1), (1,0) &SuchThat; &rho; &ge; 0
- choice between two models: constant {h<sub>0</sub>(x) = b}, linear {h<sub>1</sub>(x) = ax + b}

**Want :** value of &rho; that ties squared error measures for both models using leave-one-out cross-validation

**Note :** N = 3
- E<sub>cv</sub> = (1/N) &sum;(n=1;N){e<sub>n</sub>}
- e<sub>n</sub> = (h(x) - y)<sup>2</sup>

**Notation :**

- Let p1 = (-1,0), p2 = (&rho;, 1), p3 = (1, 0)
- Let A = {p1, p2}, B = {p1, p3}, C = {p2, p3}
- Let h<sub>0,A</sub>(x) be the hypothesis in model 0 from training on A, etc.
- Let E<sub>0,A</sub>be the validation error in model 0 on A, etc.
- Let E<sub>0</sub> be the total cross-validation error on model 0, etc.

### Constant Model
- note that training on two points in the constant model case is essentially averaging the Y coordinate.
- h<sub>0,A</sub>(x) = (p1.y+p2.y)/2 = (0+1)/2 = 1/2
- E<sub>0,A</sub>(x) = (h(x)-y)<sup>2</sup> = (1/2 - p3.y)<sup>2</sup> = (1/2 - 0)<sup>2</sup> = 1/4
- h<sub>0,B</sub>(x) = (p1.y+p3.y)/2 = (0 + 0)/2 = 0
- E<sub>0,B</sub> = (h(x)-p2.y)<sup>2</sup> = (0-1)<sup>2</sup> = 1
- h<sub>0,C</sub>(x) = (p2.y+p3.y)/2 = (1+0)/2 = 1/2
- E<sub>0,C</sub> = (h(x)-p1.y)<sup>2</sup> = (1/2-0)<sup>2</sup> = 1/4
- E<sub>0</sub> = (1/3) &times; (1/4 + 1 + 1/4) = 1/2

### Linear Model
- note that training on two points in the linear model case is finding the line that passes through the two points, with the equation of a line being y = mx + b where b is the y-intercept and m = (y2-y1)/(x2-x1)
- h<sub>1,A</sub>(x):
    - m = (p2.y-p1.y)/(p2.x-p1.x) = (1 - 0)/(&rho; - (-1)) = 1/(&rho; + 1)
    - y = mx + b &rArr; 0 = -1 &times; (1/(&rho; + 1) + b &rArr; b = 1/(&rho; + 1)
    - = (x+1)/(&rho; + 1)
- E<sub>1,A</sub> = (h(p3.x) - p3.y)<sup>2</sup> = ((1+1)/(&rho; + 1) - 0)<sup>2</sup> = (2/(&rho; + 1))<sup>2</sup> = 4/((&rho; + 1)<sup>2</sup>)
- h<sub>1,B</sub>(x):
    - m = (p3.y-p1.y)/(p3.x-p1.x) = (0-0)/(1 - (-1)) = 0/2 = 0
    - y = mx + b &rArr; p1.y = 0 &times; p1.x + b &rArr; 0 = 0 &times; -1 + b &rArr; b = 0
    - = 0
- E<sub>1,B</sub> = (h(p2.x) - p2.y)<sup>2</sup> = (0 - 1)<sup>2</sup> = 1
- h<sub>1,C</sub>(x):
    - m = (p3.y-p2.y)/(p3.x-p2.x) = (0-1)/(1-&rho;) = -1/(1-&rho;) = 1/(&rho;-1)
    - p3.y = m &times; p3.x + b &rArr; 0 = 1/(&rho;-1) + b &rArr; b = 1/(1-&rho;)
    - = (x-1)/(&rho;-1) 
- E<sub>1,C</sub> = (h(p1.x)-p1.y)<sup>2</sup> = ((-1 - 1)/(&rho;-1) - 0)<sup>2</sup> = 4/((&rho;-1)<sup>2</sup>)
- E<sub>1</sub> = (1/3) &times; ( 1 + (4((&rho; + 1)<sup>2</sup> + (&rho; - 1)<sup>2</sup>))/((&rho; + 1)<sup>2</sup>(&rho; - 1)<sup>2</sup>)))


### Equating errors

E<sub>0</sub> = E<sub>1</sub>

&rArr; 1/2 = (1/3) &times; ( 1 + (4((&rho; + 1)<sup>2</sup> + (&rho; - 1)<sup>2</sup>))/((&rho; + 1)<sup>2</sup>(&rho; - 1)<sup>2</sup>)))

&rArr; 3/2 = 1 + 4((&rho; + 1)<sup>2</sup> + (&rho; - 1)<sup>2</sup>))/((&rho; + 1)<sup>2</sup>(&rho; - 1)<sup>2</sup>))

&rArr; 1/2 = 4((&rho; + 1)<sup>2</sup> + (&rho; - 1)<sup>2</sup>))/((&rho; + 1)<sup>2</sup>(&rho; - 1)<sup>2</sup>))

&rArr; 1/8 = ((&rho; + 1)<sup>2</sup> + (&rho; - 1)<sup>2</sup>))/(((&rho; + 1)(&rho; - 1))<sup>2</sup>)

&rArr;  1/8 = (2&rho;<sup>2</sup> + 2)/((&rho;<sup>2</sup> - 1)<sup>2</sup>)

&rArr;  1/8 = 2(&rho;<sup>2</sup> + 1)/(&rho;<sup>4</sup> - 2&rho;<sup>2</sup> + 1)

&rArr;  1/16 = (&rho;<sup>2</sup> + 1)/(&rho;<sup>4</sup> - 2&rho;<sup>2</sup> + 1)

&rArr;  16(&rho;<sup>2</sup> + 1) = &rho;<sup>4</sup> - 2&rho;<sup>2</sup> + 1

&rArr;  16&rho;<sup>2</sup> + 16 = &rho;<sup>4</sup> - 2&rho;<sup>2</sup> + 1

&rArr;  &rho;<sup>4</sup> - 18&rho;<sup>2</sup> - 15 = 0

(using the quadratic formula...)

&rArr; x<sup>2</sup> = (-b  &plusmn; sqrt(b<sup>2</sup> - 4ac))/2a = (18 &plusmn; sqrt(324 + 60))/2 =  9 &plusmn; 4 &times; sqrt(6)

Since we're interested in real solutions and 4 &times; sqrt(6) &ge; 9:

**x = sqrt(9 + 4 &times; sqrt(6))**


## PLA vs SVM

**Given :**
- target function f as line defined on [-1,1] x [-1,1] where anything above the line maps to 1 and -1 otherwise
- we wish to compare the performance of PLA vs SVM
- I've coded the PLA class in pla.py, and have coded the SVM class in svm.py using the cvxopt library for quadratic programming

In [2]:
from pla import PLA
from svm import SVM
from line_test import LineTest

num_exp = [1000,100]
n_vals = [10,100]
n_test = 1000 #testing sample size
for idx,n in enumerate(n_vals):
    eout_pla = np.array([])
    eout_svm = np.array([])
    sv_svm = np.array([])
    cur_exp = num_exp[idx]
    my_pla = PLA(0) #threshold for number wrong since linearly separable
    my_svm = SVM()
    for my_exp in range(cur_exp):
        my_test = LineTest(n,n_test)
        my_pla.train(my_test.train_set,my_test.train_labels)
        my_svm.train(my_test.train_set, my_test.train_labels)
        pla_err = my_pla.calc_error(my_test.test_set, my_test.test_labels)
        svm_err = my_svm.calc_error(my_test.test_set, my_test.test_labels)
        n_sv = my_svm.num_alphas
        cur_adv = svm_err < pla_err
        eout_pla = np.concatenate((eout_pla, [pla_err]))
        eout_svm = np.concatenate((eout_svm, [svm_err]))
        sv_svm = np.concatenate((sv_svm, [n_sv]))
    eavg_pla = np.average(eout_pla)
    eavg_svm = np.average(eout_svm)
    svm_adv = 100.0*np.average(np.less_equal(eout_svm, eout_pla))
    sv_avg = np.average(sv_svm)
    print("=== For a training set with N = %d with %d runs ===" % (n, cur_exp))
    print("E_out avg for PLA: %f" % eavg_pla)
    print("E_out avg for SVM: %f" % eavg_svm)
    print("SVM improves on performance over PLA %f%% of the time" % svm_adv)
    print("Avg number of support vectors: %f" % sv_avg)
    
        
    

=== For a training set with N = 10 with 1000 runs ===
E_out avg for PLA: 0.089959
E_out avg for SVM: 0.079959
SVM improves on performance over PLA 68.800000% of the time
Avg number of support vectors: 2.168000
=== For a training set with N = 100 with 100 runs ===
E_out avg for PLA: 0.012510
E_out avg for SVM: 0.008450
SVM improves on performance over PLA 79.000000% of the time
Avg number of support vectors: 2.580000
